<a href="https://colab.research.google.com/github/fasthill/ML-DL-study-alone/blob/main/5-1%20%EA%B2%B0%EC%A0%95%20%ED%8A%B8%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Analyse from LightGBM

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import datetime
import os, re

In [2]:
from xgboost import XGBClassifier
import xgboost
from lightgbm import LGBMClassifier
import lightgbm
import joblib

In [3]:
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer

In [4]:
from sklearn.linear_model import LogisticRegression, SGDClassifier, SGDRegressor
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import make_scorer

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [5]:
import pickle

# write list, dictionary to pickle
def save_to_pickle(path, filename):
    open_file = open(path, "wb")
    pickle.dump(filename, open_file)
    open_file.close()

# read list, dictionary from pickle
def load_from_pickle(path):
    open_file = open(path, "rb")
    loaded_file = pickle.load(open_file)
    open_file.close()
    return loaded_file

In [6]:
import csv

# write list, dictionary to csv
# path = './xxx/', my_dict = filename

def save_dict_to_csv(path, my_dict):
    fields=my_dict.keys()
    values = my_dict.values()
    pd.DataFrame([fields, values], index=['parameter','value']).T.to_csv(path)
#     df = pd.DataFrame.from_dict(my_dict, orient='index') 
#     df.to_csv (path, index=False, header=True)  
    
def save_list_to_csv(path, my_list):
    df = pd.DataFrame(my_list, columns=['columns'])
    df.to_csv (path, index=False, header=True) 
    
# def load_dict_from_csv(path):
#     df = pd.read_csv(path, header=None)
#     my_dict = df.to_dict()
#     return my_dict

In [7]:
def predict_p(test_target, y_predict_list): 
    ps = precision_score(test_target, y_predict_list)
    rs = recall_score(test_target, y_predict_list)
    fs = f1_score(test_target, y_predict_list)
    roc = roc_auc_score(test_target, y_predict_list)
    collect_list = [ps, rs, fs, roc]
    return collect_list

In [8]:
def make_df_from_estimator(estimator, num):
    df_t = pd.DataFrame.from_dict(estimator, orient='index')
    df_t.columns = [f'iter_{num}']
    df_t.index.name = 'parameter'
    return df_t

In [9]:
def calc_results(lgbm, model, train_scaled, val_scaled, test_scaled, train_target, val_target, test_target,
                 test_scaled1, test_scaled2, test_scaled3, test_target1, test_target2, test_target3):
# model = lgbmgs.best_estimator_  # 최적의 파라미터로 모델 생성
    y_predict = model.predict(test_scaled)
    result_dict= {}
    result_dict['best_score'] = lgbm.best_score_ 
    result_dict['best_index'] = lgbm.best_index_
    result_dict['acc_train'] = model.score(train_scaled, train_target)
    result_dict['acc_val'] = model.score(val_scaled, val_target)
    result_dict['acc_test'] = model.score(test_scaled, test_target)
    result_dict['precision'] = precision_score(test_target, y_predict)
    result_dict['recall'] = recall_score(test_target, y_predict)
    result_dict['f1score'] = f1_score(test_target, y_predict)
    result_dict['roc'] = roc_auc_score(test_target, y_predict)
    cm = confusion_matrix(test_target, y_predict)
    result_dict['tn'] = cm[0,0]
    result_dict['fp'] = cm[0,1]
    result_dict['fn'] = cm[1,0]
    result_dict['tp'] = cm[1,1]
#     result_dict['precision_neg'] = cm[0,0] / (cm[0,0] + cm[1,0])
    #------------------------------------
    y_predict = model.predict(test_scaled1)
    result_dict['acc_test1'] = model.score(test_scaled1, test_target1)
    result_dict['precision1'] = precision_score(test_target1, y_predict)
    result_dict['recall1'] = recall_score(test_target1, y_predict)
    result_dict['f1score1'] = f1_score(test_target1, y_predict)
    result_dict['roc1'] = roc_auc_score(test_target1, y_predict)
    cm = confusion_matrix(test_target1, y_predict)
    result_dict['tn1'] = cm[0,0]
    result_dict['fp1'] = cm[0,1]
    result_dict['fn1'] = cm[1,0]
    result_dict['tp1'] = cm[1,1]
#     result_dict['precision_neg'] = cm[0,0] / (cm[0,0] + cm[1,0])
    #------------------------------------
    y_predict = model.predict(test_scaled2)
    result_dict['acc_test2'] = model.score(test_scaled2, test_target2)
    result_dict['precision2'] = precision_score(test_target2, y_predict)
    result_dict['recall2'] = recall_score(test_target2, y_predict)
    result_dict['f1score2'] = f1_score(test_target2, y_predict)
    result_dict['roc2'] = roc_auc_score(test_target2, y_predict)
    cm = confusion_matrix(test_target2, y_predict)
    result_dict['tn2'] = cm[0,0]
    result_dict['fp2'] = cm[0,1]
    result_dict['fn2'] = cm[1,0]
    result_dict['tp2'] = cm[1,1]
#     result_dict['precision_neg'] = cm[0,0] / (cm[0,0] + cm[1,0])
    #------------------------------------
    y_predict = model.predict(test_scaled3)
    result_dict['acc_test3'] = model.score(test_scaled3, test_target3)
    result_dict['precision3'] = precision_score(test_target3, y_predict)
    result_dict['recall3'] = recall_score(test_target3, y_predict)
    result_dict['f1score3'] = f1_score(test_target3, y_predict)
    result_dict['roc3'] = roc_auc_score(test_target3, y_predict)
    cm = confusion_matrix(test_target3, y_predict)
    result_dict['tn3'] = cm[0,0]
    result_dict['fp3'] = cm[0,1]
    result_dict['fn3'] = cm[1,0]
    result_dict['tp3'] = cm[1,1]
#     result_dict['precision_neg'] = cm[0,0] / (cm[0,0] + cm[1,0])
    return result_dict

In [10]:
def save_best_results(com_name, model, scaler, columns, df_base):
    joblib.dump(model, f'{directory_for_model}{com_name}/best_model.pkl') # estimaor 저장
    joblib.dump(scaler, f'{directory_for_model}{com_name}/best_scaler.pkl') # scaler 저장
    save_to_pickle(f'{directory_for_model}{com_name}/best_model_p.pkl', model)
    save_to_pickle(f'{directory_for_model}{com_name}/best_scaler_p.pkl', scaler)
    save_to_pickle(f'{directory_for_model}{com_name}/best_columns.pkl', new_col)
    save_list_to_csv(f'{directory_for_model}{com_name}/best_columns.csv', new_col)
    df_base.to_pickle(f'{directory_for_model}{com_name}/best_result.pkl')
    return

In [11]:
def add_dict(*dicts):
    dsum = {}
    d_k = []
    d_v = []
    for dic in dicts:
        d_k.extend(list(dic.keys()))  # sum keys list
        d_v.extend(list(dic.values()))  # sum values list
    for i in range(len(d_k)):
        dsum[d_k[i]] = d_v[i]
    return dsum

In [54]:
def df_combine_sorted(df_base_l, df_params_l):
    o_num = list(df_base_l.index).index('best_score')
    df1 = df_base.iloc[:o_num, :]
    df2 = df_base.iloc[o_num:, :]
    df3 = pd.concat([df1, df_params_l], axis=0)
    df3.sort_index(axis=0, inplace=True)
    df_sorted = pd.concat([df3, df2], axis=0)
    return df_sorted

In [12]:
import sys, os

module_path = os.path.abspath(os.path.join('..')) # 현재 폴더로 이동
if module_path+"\\data\\base_data\\common_data" not in sys.path:
    sys.path.append(module_path+"\\data\\base_data\\common_data") #  공통으로 사용하는 각종 리스트, 코드 등 
    
import common_data as cd

In [13]:
code = cd.code_all # 전체 회사 코드
code = {'006400': ['삼성SDI', 'sdi']}

In [14]:
col_inv1 = ['retail_1', 'foreigner_1', 'institution_1', 'financial_1', 'invtrust_1', 'pension_1', 
#             'privequity_1', 'bank_1', 'insurance_1', 'financeetc_1', 'corporateetc_1', 
            'privequity_1',  'insurance_1', 'corporateetc_1', # bank_1, 'financeetc_1 제외
            'foreigneretc_1']
col_inv2 = ['retail_2', 'foreigner_2', 'institution_2', 'financial_2', 'invtrust_2', 'pension_2',
#             'privequity_2', 'bank_2', 'insurance_2', 'financeetc_2', 'corporateetc_2', 
            'privequity_2', 'insurance_2', 'corporateetc_2', # bank_2, 'financeetc_2 제외
            'foreigneretc_2']
col_his1 = ['open_1', 'high_1', 'low_1', 'close_1', 'vol_1']
col_his2 = ['open_2', 'high_2', 'low_2', 'close_2', 'vol_2']
col_cr = ['weekday', 'cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20']
col_common1 = ["dji_cr", "dji_f_cr", "dxy_cr", "ixic_f_cr", "bond_kor_10_cr", "bond_kor_2_cr", "kosdaq_cr", "kospi_cr", 
         "krw_cr", "ixic_cr", "spx_f_cr", "sox_cr", "spx_cr", "bond_usa_10_cr", "bond_usa_2_cr", "bond_usa_3m_cr", 
         "vix_cr", "wti_cr", "spsy_cr", "spny_cr", "spxhc_cr", "splrcd_cr", "splrci_cr", "splrcu_cr", "splrcs_cr",
         "splrct_cr", "splrcl_cr", "splrcm_cr", "ixbk_cr", "ixfn_cr", "ixid_cr", "ixis_cr", "ixk_cr", "ixtr_cr",
         "ixut_cr", "nbi_cr", "bkx_cr"]
col_common2 = ["dji_cr_2", "dji_f_cr_2", "dxy_cr_2", "ixic_f_cr_2", "bond_kor_10_cr_2", "bond_kor_2_cr_2", "kosdaq_cr_2", "kospi_cr_2",
         "krw_cr_2", "ixic_cr_2", "spx_f_cr_2", "sox_cr_2", "spx_cr_2", "bond_usa_10_cr_2", "bond_usa_2_cr_2", "bond_usa_3m_cr_2",
         "vix_cr_2", "wti_cr_2", "spsy_cr_2", "spny_cr_2", "spxhc_cr_2", "splrcd_cr_2", "splrci_cr_2", "splrcu_cr_2",
         "splrcs_cr_2", "splrct_cr_2", "splrcl_cr_2", "splrcm_cr_2", "ixbk_cr_2", "ixfn_cr_2", "ixid_cr_2",
         "ixis_cr_2", "ixk_cr_2", "ixtr_cr_2", "ixut_cr_2", "nbi_cr_2", "bkx_cr_2"]
col_futures = ['ixic_f_cr', 'ixic_f_cr_2', 'spx_f_cr', 'spx_f_cr_2', 'dji_f_cr', 'dji_f_cr_2',
           'wti_cr','wti_cr_2', 'dxy_cr', 'dxy_cr_2', 'bond_usa_10_cr', 'bond_usa_10_cr_2' ]
column_o = col_inv1 + col_common1 + col_his1 + col_inv2 + col_common2 + col_his2 + col_cr

col_except_futures = [item for item in column_o if item not in col_futures]

new_col = col_except_futures.copy()

# bank, financeetc는 결측치가 많아서 사용하지 않음.
# df.drop(['bank_1', 'bank_2', 'financeetc_1', 'financeetc_2'], axis=1, inplace=True)   

# col_futures : futures는 당일 종료가 되지 않는 data이므로 제외

In [15]:
# 최초의 empty df 생성
df_base = pd.DataFrame(pd.Series([],dtype=pd.StringDtype(), name='parameter')).set_index('parameter')
df_params = pd.DataFrame(pd.Series([],dtype=pd.StringDtype(), name='parameter')).set_index('parameter')
iter = 0

In [16]:
#### 신규 분석 데이터는 아래 것으로 사용

com_name = list(code.values())[0][1]

directory_for_ml = '../data/data_for_ml/predict/'
fname = f'df_{com_name}_combine.pkl'
f_name = directory_for_ml + fname
df_o = pd.read_pickle(f_name) 
df_o = df_o.iloc[:-1] # /predict/를 사용할 경우 마지막 prediction data 제외

In [17]:
len(df_o)

311

In [18]:
# 반복 작업시 여기서 부터 진행 (feature importance로 선정된 새로운 column으로)

df = df_o[new_col]  # 새롭게 선정된 column으로 진행

# train, val,: 8, test: 2
split_ratio = 0.8
split_n = int(len(df)*split_ratio)

test_interval = int((len(df) - split_n)/3)
data = df.iloc[:split_n, :-5]
target = df.iloc[:split_n, -5]
test_input = df.iloc[split_n:, :-5]
test_target = df.iloc[split_n:, -5]
test_input1 = df.iloc[split_n:split_n+test_interval, :-5]
test_input2 = df.iloc[split_n+test_interval: split_n+test_interval*2, :-5]
test_input3 = df.iloc[split_n+test_interval*2:, :-5]
test_target1 = df.iloc[split_n:split_n+test_interval, -5]
test_target2 = df.iloc[split_n+test_interval: split_n+test_interval*2, -5]
test_target3 = df.iloc[split_n+test_interval*2:, -5]

train_input, val_input, train_target, val_target = \
     train_test_split(data, target, random_state=42, test_size=0.2, stratify=target)

scaler = None
scaler = StandardScaler()
scaler.fit(train_input)
train_scaled = scaler.transform(train_input)
val_scaled = scaler.transform(val_input)
test_scaled = scaler.transform(test_input)
test_scaled1 = scaler.transform(test_input1)
test_scaled2 = scaler.transform(test_input2)
test_scaled3 = scaler.transform(test_input3)

In [110]:
search_space = {
    'boosting_type' : ['gbdt', 'dart', ],# ['gbdt', 'dart', 'goss'],
    'learning_rate': (0.0001, 0.004, 'log-uniform'),
    'num_leaves': (3, 20),      
    'max_depth': (0, 50), #[-1], #(0, 50),
    'min_child_samples': (5, 40),
#     'min_child_weight': (0.001, 10.0), # 값을 변동시 같은 값이 최적값으로 선정되더라고 precision이 틀림. 반드시 값을 바꾸면서 진행해야 함.
#     'min_split_loss': (0, 5), 
    'max_bin': (100, 1000), # 사용시 0.75%로 올라감 default: 255
#     'max_cat_threshold' : (1, 100),
    'subsample': (0.005, 1.0, 'uniform'), # == bagging_fraction
#     'subsample_freq': (1, 10), # mobis에서 적용시 값이 많이 올라감. default:0
    'colsample_bytree': (0.005, 1.0, 'uniform'),
    'subsample_for_bin': (100000, 500000),
    'scale_pos_weight' : (0.2, 2.0, 'uniform'),
#     'scale_pos_weight': np.arange(0.2, 2, 0.1) ,
    'n_estimators': (700, 5000),
    'reg_alpha' : (0.0, 4.0, 'uniform'), # =='lambda_l1': [0, 5], # default 0
    'reg_lambda' : (0.0, 4.0, 'uniform'), # == lambda_l2': [0, 5], #default 0    
#     'reg_alpha' : np.arange(0, 4, 0.1),
#     'reg_lambda' : np.arange(0, 4, 0.1),
    'force_col_wise': ['true'], 
    'importance_type': ['split'], # ['gain'], default: split
    'boost_from_average' : [True, False],
    'objective': ['binary'],
    'metric': ['binary_logloss'],
}

params_space = search_space.copy() # for Bayesian Optimization

## BayesSearchCV

In [111]:
param_grid = {
    'cv' : 3,
#     'cv' : None,
#     'scoring' : None,
#     'scoring' : 'f1',
#     'scoring' : 'roc_auc',
    'scoring' : 'precision',
#     'scoring' : 'average_precision',
#     'scoring' : 'accuracy',
    'num_col' : len(new_col),
    'iterations' : 50,  # default : 50, number of parameter settings.
             }

In [112]:
# directory_for_model = '../data/data_for_ml/model/model/0. test/'
directory_for_model = '../data/data_for_ml/model/model/'

In [113]:
# directory가 없으면 만드는 과정
if not os.path.exists(directory_for_model+com_name):
    os.makedirs(directory_for_model+com_name)

iter = iter + 1

lgbm = None
lgbmBS = None
gsbs = 'bs'

lgbmBS = BayesSearchCV( estimator = lightgbm.LGBMClassifier(verbose=0, random_state=42),
                       search_spaces = params_space,
                       scoring = param_grid['scoring'],
                       cv = StratifiedKFold( n_splits=param_grid['cv'],
                                             shuffle=True,
                                             random_state=42 ),
                       n_jobs = -1, # 자동 검색 적용
                       n_iter = param_grid['iterations'],   
                       verbose = 0, refit = True, random_state = 42 
                      )

print("*** after lgbm BS ******")
lgbmBS.fit(train_scaled, train_target, eval_metric = 'logloss') 

# save model
stamp = datetime.datetime.today().isoformat() # 파일명 끝에 생성날짜 시간 추가
dt = re.sub(r'[-:T]', '', stamp[5:16])
dt = f'{dt[:4]}_{dt[4:]}'

df_estimator = make_df_from_estimator(lgbmBS.best_estimator_.get_params(), iter)
df_estimator.sort_index(inplace=True) # alphabet 순으로 보기 편하게

# metrics accuracy,,,, 3단계 precision 등까지. dictionary
result_dict = calc_results(lgbmBS, lgbmBS.best_estimator_, 
                           train_scaled, val_scaled, test_scaled,  
                           train_target, val_target, test_target,
                           test_scaled1, test_scaled2, test_scaled3,  
                           test_target1, test_target2, test_target3
                          )

df_grid = make_df_from_estimator(param_grid, iter) # gridcv parameter
df_grid.sort_index(inplace=True) # alphabet 순으로 보기 편하게
df_result = make_df_from_estimator(result_dict, iter)  # dict 를 df로
df_concat = pd.concat([df_grid, df_estimator, df_result])

df_base = pd.merge(df_base,df_concat, how='outer', left_index=True, right_index=True)

val_test = df_concat.loc['acc_val'].iloc[0]
acc_test = df_concat.loc['acc_test'].iloc[0]
precision = df_concat.loc['precision'].iloc[0]
f1score = df_concat.loc['f1score'].iloc[0]

params_search = add_dict(param_grid, search_space)
df_params_search = make_df_from_estimator(params_search, iter)
df_params = pd.merge(df_params,df_params_search, how='outer', left_index=True, right_index=True)
  
print("******* No.{} BS Process is Done! ********".format(iter))
    
print("**** End of BayesSearchCV Process ****")

*** after lgbm BS ******
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

******* No.12 BS Process is Done! ********
**** End of BayesSearchCV Process ****


In [114]:
df_one = df_combine_sorted(df_base, df_params)
df_one.head(60)

,iter_1,iter_2,iter_3,iter_4,iter_5,iter_6,iter_7,iter_8,iter_9,iter_10,iter_11,iter_12
parameter,,,,,,,,,,,,
boost_from_average,"[True, False]","[True, False]","[True, False]","[True, False]","[True, False]","[True, False]","[True, False]","[True, False]","[True, False]","[True, False]","[True, False]","[True, False]"
boost_from_average,False,False,True,True,True,True,True,False,True,False,True,False
boosting_type,"[gbdt, dart]","[gbdt, dart]","[gbdt, dart]","[gbdt, dart]","[gbdt, dart]","[gbdt, dart]","[gbdt, dart]","[gbdt, dart]","[gbdt, dart]","[gbdt, dart]","[gbdt, dart]","[gbdt, dart]"
boosting_type,dart,dart,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,dart
class_weight,None,None,None,None,None,None,None,None,None,None,None,None
colsample_bytree,"(0.005, 1.0, uniform)","(0.005, 1.0, uniform)","(0.005, 1.0, uniform)","(0.003, 1.0, uniform)","(0.01, 1.0, uniform)","(0.0001, 1.0, uniform)","(0.001, 1.0, uniform)","(0.005, 1.0, uniform)","(0.005, 1.0, uniform)","(0.005, 1.0, uniform)","(0.005, 1.0, uniform)","(0.005, 1.0, uniform)"
colsample_bytree,0.005,0.175842,0.391796,0.003,0.01,0.0001,0.001,0.05301,1.0,0.239176,0.106201,0.005
cv,3,3,5,3,3,3,3,3,3,3,3,3
cv,3,3,5,3,3,3,3,3,3,3,3,3


In [115]:
df_one.iloc[52:70, :]

,iter_1,iter_2,iter_3,iter_4,iter_5,iter_6,iter_7,iter_8,iter_9,iter_10,iter_11,iter_12
parameter,,,,,,,,,,,,
best_score,0.763889,0.631313,0.742222,0.727918,0.72381,0.755981,0.755981,0.6875,0.711501,0.708333,0.706548,0.763889
best_index,47.0,36.0,31.0,17.0,34.0,37.0,37.0,39.0,24.0,26.0,10.0,47.0
acc_train,0.656566,0.727273,0.656566,0.681818,0.641414,0.691919,0.691919,0.69697,0.621212,0.681818,0.621212,0.656566
acc_val,0.54,0.7,0.54,0.58,0.5,0.58,0.58,0.5,0.54,0.56,0.52,0.54
acc_test,0.603175,0.52381,0.539683,0.555556,0.587302,0.571429,0.571429,0.571429,0.571429,0.571429,0.571429,0.603175
precision,0.8,0.464286,0.428571,0.5,0.75,0.571429,0.571429,0.6,0.666667,0.571429,0.6,0.8
recall,0.142857,0.464286,0.107143,0.178571,0.107143,0.142857,0.142857,0.107143,0.071429,0.142857,0.107143,0.142857
f1score,0.242424,0.464286,0.171429,0.263158,0.1875,0.228571,0.228571,0.181818,0.129032,0.228571,0.181818,0.242424
roc,0.557143,0.517857,0.496429,0.517857,0.539286,0.528571,0.528571,0.525,0.521429,0.528571,0.525,0.557143


In [116]:
# search_space, parameters and results save
df_one.to_csv(f'{directory_for_model}{com_name}/params_{gsbs}_df_{dt}.csv')
df_one.to_pickle(f'{directory_for_model}{com_name}/params_{gsbs}_df_{dt}.pkl')

In [117]:
# parameter and results save
df_base.to_csv(f'{directory_for_model}{com_name}/lgbm_{gsbs}_df_{dt}.csv')
df_base.to_pickle(f'{directory_for_model}{com_name}/lgbm_{gsbs}_df_{dt}.pkl')

# search_space, parameters save
df_params.to_csv(f'{directory_for_model}{com_name}/params_{gsbs}_df_{dt}.csv')
df_params.to_pickle(f'{directory_for_model}{com_name}/params_{gsbs}_df_{dt}.pkl')

In [42]:
# save model
# 필요시 저장. 가장 좋은 결과만 저장하고자할 때는 밑의 save_best_results만 진행하면 됨.
joblib.dump(lgbmBS.best_estimator_, f'{directory_for_model}{com_name}/estimator_{gsbs}_{dt}_v{iter}.pkl') # bayessearchcv 저장
save_to_pickle(f'{directory_for_model}{com_name}/model_{gsbs}_{dt}_v{iter}_p.pkl', lgbmBS.best_estimator_)
joblib.dump(scaler, f'{directory_for_model}{com_name}/scaler_{gsbs}_{dt}_v{iter}.pkl') # scaler 저장
save_to_pickle(f'{directory_for_model}{com_name}/columns_{gsbs}_{dt}_{len(new_col)}_{round(precision*100):2d}%_{round(f1score*100):2d}%_ver{iter}.pkl', new_col)
save_list_to_csv(f'{directory_for_model}{com_name}/columns_{gsbs}_{dt}_{len(new_col)}_{round(precision*100):2d}%_{round(f1score*100):2d}%_ver{iter}.csv', new_col)

In [118]:
# 최종 가장 좋은 결과 저장
save_best_results(com_name, lgbmBS.best_estimator_, scaler, new_col, df_base)

In [119]:
print(lgbmBS.best_estimator_)

LGBMClassifier(boost_from_average=False, boosting_type='dart',
               colsample_bytree=0.005, force_col_wise='true',
               learning_rate=0.0031006343684312075, max_bin=258, max_depth=4,
               metric='binary_logloss', min_child_samples=28, n_estimators=2763,
               num_leaves=16, objective='binary', random_state=42,
               reg_alpha=3.271685107079131, reg_lambda=3.300273289696091,
               scale_pos_weight=0.6928487757652826,
               subsample=0.09810428848269573, subsample_for_bin=348648,
               verbose=0)


In [120]:
model = lgbmBS.best_estimator_
feature_df = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), 
                      index=data.columns, columns=['importance']).sort_values(by='importance', 
                                                                              ascending=False)

In [121]:
len(feature_df)

93

In [122]:
feature_df.tail(70)

,importance
nbi_cr,80.714988
spxhc_cr_2,80.682913
bond_kor_10_cr_2,80.573331
open_2,78.557979
ixtr_cr,74.934368
...,...
insurance_2,4.101092
high_1,2.906532
weekday,2.725675
retail_1,2.400639


In [ ]:
new_col = list(feature_df.index[:35]) +  ['cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20']